# INIT

Start


In [ ]:
import sys
import pprint
from pathlib import Path

sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

import ipytest
ipytest.autoconfig()

from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, base64_to_hex
from utils.logger import get_logger

from contracts.farm_contract import FarmContract

from multiversx_sdk import ProxyNetworkProvider, ApiNetworkProvider

logger = get_logger("manual_interactor")

SIMULATOR_URL = "http://localhost:8085"
SIMULATOR_API = "http://localhost:3001"
GENERATE_BLOCKS_URL = f"{SIMULATOR_URL}/simulator/generate-blocks"
PROJECT_ROOT = Path.cwd().parent.parent
proxy = ProxyNetworkProvider(SIMULATOR_URL)
api = ApiNetworkProvider(SIMULATOR_API)

context = Context()
context.network_provider.proxy = proxy
context.network_provider.api = api

In [ ]:
farm_contract: FarmContract = context.get_contracts(config.FARMS_V2)[0]

pprint.pprint(farm_contract.get_config_dict())

Permission Hub

In [ ]:
from contracts.permissions_hub_contract import PermissionsHubContract

def deploy_permissions_hub():
    permissions_hub_contract = PermissionsHubContract("")
    _, address = permissions_hub_contract.contract_deploy(context.deployer_account, context.network_provider.proxy, 
                                            "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.2-rc2/permissions-hub.wasm",
                                            [])
    permissions_hub_contract.address = address
    return permissions_hub_contract

In [ ]:
def set_permissions_hub(permissions_hub_contract: PermissionsHubContract):
    farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)

COLLECT USERS

In [ ]:
from typing import List
from utils.utils_scenarios import FetchedUser, collect_farm_contract_users
from multiversx_sdk import ApiNetworkProvider

context = Context()
mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
fetched_users = collect_farm_contract_users(400, farm_contract.address, farm_contract.farmingToken, farm_contract.farmToken,
                                            mainnet_api, context.network_provider.proxy)

users_farm_tokens: List[FetchedUser] = fetched_users.get_users_with_farm_tokens()
users_farming_tokens: List[FetchedUser] = fetched_users.get_users_with_farming_tokens()
users_both_tokens: List[FetchedUser] = fetched_users.get_users_with_both_tokens()

for user in users_farm_tokens:
    print(user.address)
print("////////////")
for user in users_farming_tokens:
    print(user.address)
print("////////////")
for user in users_both_tokens:
    print(user.address)

# Reset context
context.network_provider.proxy = proxy
context.network_provider.api = api

CHAIN CONFIG SETUP

In [ ]:
import json
from typing import Any
from utils.utils_chain import WrapperAddress

USERS = ["erd1qlylg4zrtprcw8de4q9ltrv0gtd8rwxklgsl6w7scy97r9ettkjsgzkrma", # user with farm position 
         "erd1670736z99swpr7z7xm6p0uqkymjdyhjv2dxl49n84f7znf94v8zsw6a83n", # user with farming tokens
         "erd1qehazazmsdaftaxhv553utjqugtkfp8drdvcqvw8zftr2v93j47skslkuw" # user with both tokens
        ]
def load_accounts_state(project_root: Path, addresses: list[str]) -> list[dict[str, Any]]:
    states = []
    
    for address in addresses:
        print(f"Loading state for {address}")
        user_path = f"0_{address}_0_chain_config_state.json"
        system_account_path = f"0_system_account_state_{address}.json"
        
        user_file = project_root / "states" / user_path
        system_file = project_root / "states" / system_account_path
        
        if user_file.exists():
            with open(user_file, "r") as file:
                user_state = json.load(file)
                if user_state:
                    print(f"Found {user_file.name}")
                    states.append(user_state)
                
        if system_file.exists():
            with open(system_file, "r") as file:
                system_state = json.load(file)
                if system_state:
                    print(f"Found {system_file.name}")
                    states.append(system_state)
            
    return states
    

def apply_states(proxy: ProxyNetworkProvider, states: list[dict[str, Any]]):
    for state in states:
        proxy.do_post(f"{SIMULATOR_URL}/simulator/set-state", state)


def load_and_apply_state(proxy: ProxyNetworkProvider, project_root: Path, owner: str, users: list[str]):
    # Load and set state for all keys
    with open(project_root / "states" / "0_all_all_keys.json", "r") as file:
        retrieved_state = json.load(file)
        apply_states(proxy, [retrieved_state])

    # Load owner and users state
    accounts = [owner]
    accounts.extend(users)
    states = load_accounts_state(project_root, accounts)
    apply_states(proxy, states)
        

def setup_chain_sim():
    # generate blocks to pass an epoch and the smart contract deploys to be enabled
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/5", {})

    load_and_apply_state(proxy, PROJECT_ROOT,
                         context.deployer_account.address.bech32(),
                         USERS)


def advance_blocks(number_of_blocks: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_blocks}", {})

def advance_epoch(number_of_epochs: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_epochs * 20}", {})


def users_init() -> list[Account]:
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    users = []
    for user in USERS:
        user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
        user_account.address = WrapperAddress(user)
        user_account.sync_nonce(context.network_provider.proxy)
        users.append(user_account)

    return users

CHAIN SIMULATOR STACK

In [ ]:
import subprocess
from time import sleep

CS_DOCKER_PATH = Path.home() / "projects/testing/full-stack-docker-compose/chain-simulator"

def start_chain_sim_stack():
    # stop first in case one is already running
    print("Initializing chain simulator...")
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH, stdout=subprocess.DEVNULL, 
        stderr=subprocess.PIPE)
    p.wait()
    
    p = subprocess.Popen(["docker", "compose", "up", "-d"], cwd = CS_DOCKER_PATH, stdout=subprocess.DEVNULL, 
        stderr=subprocess.PIPE)
    sleep(60)
    return p

def stop_chain_sim_stack(p):
    print("Stopping chain simulator...")
    p.terminate()
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH, stdout=subprocess.DEVNULL, 
        stderr=subprocess.PIPE)
    p.wait()
    _ = subprocess.run(["docker", "system", "prune", "-f"], cwd = CS_DOCKER_PATH, stdout=subprocess.DEVNULL, 
        stderr=subprocess.PIPE)
    print("Chain simulator stopped.")

Farm upgrade

In [ ]:
def farm_upgrade():
    tx_hash = farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.3.1-rc1/farm-with-locked-rewards.wasm", [], True)

    advance_blocks(1)
    tx_hash = farm_contract.resume(context.deployer_account, context.network_provider.proxy)
    advance_blocks(1)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(farm_contract.address)).code_hash
    assert base64_to_hex(code_hash) == "78c4451db7425405e85b638a556043c143878ee1f3fc542f424d8f9aba1a61ee"

Enter farm consolidated

In [ ]:
from utils.utils_chain import Account, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_for_user_consolidated(user_account):

    user_account.sync_nonce(context.network_provider.proxy)
    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > 0:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farm tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance,
                        farm_contract.farmToken, farm_tk_nonce, farm_tk_balance)

    tx_hash = farm_contract.enterFarm(context.network_provider, user_account, event)
    return tx_hash

In [ ]:
from utils.utils_chain import Account, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_for_user(user_account):

    user_account.sync_nonce(context.network_provider.proxy)
    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > 0:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance,
                        "", 0, 0)

    tx_hash = farm_contract.enterFarm(context.network_provider, user_account, event)
    return tx_hash

Exit farm

In [ ]:
from contracts.farm_contract import ExitFarmEvent

def exit_farm_user(user_account):
    user_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = ExitFarmEvent(farm_contract.farmToken, farm_tk_balance, farm_tk_nonce, '')

    tx_hash = farm_contract.exitFarm(context.network_provider, user_account, event)
    return tx_hash

Claim rewards

In [ ]:
from contracts.farm_contract import ClaimRewardsFarmEvent

def claim_rewards_user(user_account):
    user_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '')

    tx_hash = farm_contract.claimRewards(context.network_provider, user_account, event)
    return tx_hash

Claim boosted rewards

In [ ]:
def claim_boosted_rewards_user(user_account):
    user_account.sync_nonce(context.network_provider.proxy)

    event = ClaimRewardsFarmEvent(0, 0, '', user_account.address.bech32())

    tx_hash = farm_contract.claim_boosted_rewards(context.network_provider, user_account, event)
    return tx_hash

Merge positions

In [ ]:
from events.farm_events import MergePositionFarmEvent


def merge_positions(user):
    user.sync_nonce(context.network_provider.proxy)
    
    farm_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(
        farm_contract.farmToken, user.address.bech32(), context.network_provider.proxy
    )
    print(f'Found {len(tokens_in_account)} farm tokens in account')
    
    event_list = []
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            event_list.append(MergePositionFarmEvent(int(token['balance']), token['nonce'], None))
    
    tx_hash = farm_contract.mergePositions(context.network_provider, user, event_list)
    return tx_hash

Enter farm on behalf

In [ ]:
from utils.utils_chain import Account, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_on_behalf(claim_account, user_address):
    claim_account.sync_nonce(context.network_provider.proxy)
    
    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(
        farm_contract.farmingToken, claim_account.address.bech32(), context.network_provider.proxy
    )
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    
    for token in tokens_in_account:
        if int(token['balance']) > 0:
            farming_tk_balance = int(token['balance'])
            break
    
    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")
    
    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(
        farm_contract.farmToken, claim_account.address.bech32(), context.network_provider.proxy
    )
    print(f'Found {len(tokens_in_account)} farm tokens in account')
    
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break
    
    if farm_tk_balance > 0:
        event = EnterFarmEvent(
            farm_contract.farmingToken, 0, farming_tk_balance,
            farm_contract.farmToken, farm_tk_nonce, farm_tk_balance, user_address.address.bech32()
        )
    else:
        event = EnterFarmEvent(
            farm_contract.farmingToken, 0, farming_tk_balance,
            None, 0, 0, user_address.address.bech32()
        )
    
    tx_hash = farm_contract.enter_farm_on_behalf(context.network_provider, claim_account, event)
    return tx_hash

Claim rewards on behalf

In [ ]:
from utils.utils_chain import get_all_token_nonces_details_for_account

def claim_rewards_on_behalf(whitelisted_account, user_account):
    whitelisted_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, whitelisted_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '', user_account.address.bech32())

    tx_hash = farm_contract.claim_rewards_on_behalf(context.network_provider, whitelisted_account, event)
    return tx_hash

Transfer position

In [ ]:
from utils.utils_tx import multi_esdt_transfer, ESDTToken
from utils.utils_chain import Account

def transfer_position(user_account, destination):
    user_account.sync_nonce(context.network_provider.proxy)

    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farm tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > 0:
            farm_tk = token
            break
    token = ESDTToken(farm_tk['tokenIdentifier'], farm_tk['nonce'], int(farm_tk['balance']))
    print(f'Sending: {token.get_token_data()} from {user_account.address.bech32()}')
    multi_esdt_transfer(context.network_provider.proxy, 1000000, user_account, destination.address, [token])

Utils

In [ ]:
from typing import Dict, Any, List, Tuple, Optional
from dataclasses import dataclass
from enum import Enum, auto
from collections import defaultdict

class DictType(Enum):
    USER_FARM_STATS_1 = auto()
    USER_FARM_STATS_1_BEHALF = auto()
    FARM_CONTRACT_STATS_1 = auto()
    TOKEN_ATTRS_1 = auto()
    USER_FARM_STATS_2 = auto()
    USER_FARM_STATS_2_BEHALF = auto()
    FARM_CONTRACT_STATS_2 = auto()
    TOKEN_ATTRS_2 = auto()
    OP1 = auto()
    OP2 = auto()
    OP3 = auto()
    OP4 = auto()

@dataclass
class DictComparison:
    before: Dict[str, Any]
    after: Dict[str, Any]
    description: str
    phase: str

class DictCollector:
    def __init__(self):
        self.reset()

    def reset(self):
        """Reset all collections for a new scenario"""
        self.before_upgrade: Dict[DictType, List[Tuple[Dict[str, Any], str]]] = defaultdict(list)
        self.after_upgrade: Dict[DictType, List[Tuple[Dict[str, Any], str]]] = defaultdict(list)
        self.current_phase = "before"  # "before" or "after"
    
    def set_phase(self, phase: str):
        """Set the current collection phase"""
        if phase not in ["before", "after"]:
            raise ValueError("Phase must be 'before' or 'after'")
        self.current_phase = phase

    def add(self, dict_type: DictType, dict_data: Dict[str, Any], description: str = ""):
        """Add a dictionary to the current phase collection"""
        if self.current_phase == "before":
            self.before_upgrade[dict_type].append((dict_data, description))
        else:
            self.after_upgrade[dict_type].append((dict_data, description))

    def _compare_dicts(self, dict1: Any, dict2: Any) -> Tuple[bool, Optional[str]]:
        """
        Compare two objects that can be either dictionaries or lists of dictionaries.
        Returns (is_equal, difference_description)
        """
        # Handle lists of dictionaries
        if isinstance(dict1, list) and isinstance(dict2, list):
            if len(dict1) != len(dict2):
                return False, f"Different list lengths: {len(dict1)} != {len(dict2)}"
            
            for i, (item1, item2) in enumerate(zip(dict1, dict2)):
                if isinstance(item1, (dict, list)) and isinstance(item2, (dict, list)):
                    is_equal, diff = self._compare_dicts(item1, item2)
                    if not is_equal:
                        return False, f"List item {i} difference: {diff}"
                elif item1 != item2:
                    return False, f"List item {i} mismatch: {item1} != {item2}"
            return True, None

        # Handle dictionaries
        if isinstance(dict1, dict) and isinstance(dict2, dict):
            if dict1.keys() != dict2.keys():
                missing_keys = set(dict1.keys()) - set(dict2.keys())
                extra_keys = set(dict2.keys()) - set(dict1.keys())
                return False, f"Different keys. Missing: {missing_keys}, Extra: {extra_keys}"

            for key in dict1:
                if isinstance(dict1[key], (dict, list)) and isinstance(dict2[key], (dict, list)):
                    is_equal, diff = self._compare_dicts(dict1[key], dict2[key])
                    if not is_equal:
                        return False, f"Nested difference at key '{key}': {diff}"
                elif dict1[key] != dict2[key]:
                    return False, f"Value mismatch for key '{key}': {dict1[key]} != {dict2[key]}"
            return True, None

        # Handle case where types don't match
        if type(dict1) != type(dict2):
            return False, f"Type mismatch: {type(dict1)} != {type(dict2)}"

        # Handle other cases
        return dict1 == dict2, f"Value mismatch: {dict1} != {dict2}" if dict1 != dict2 else None

    def compare_all(self) -> List[str]:
        """
        Compare all collected dictionary pairs and return a list of differences found.
        """
        differences = []

        for dict_type in DictType:
            before_list = self.before_upgrade[dict_type]
            after_list = self.after_upgrade[dict_type]

            # Check if we have matching pairs
            if len(before_list) != len(after_list):
                differences.append(
                    f"{dict_type.name}: Mismatched number of collections - "
                    f"Before: {len(before_list)}, After: {len(after_list)}"
                )
                continue

            # Compare each pair
            for i, ((before_dict, before_desc), (after_dict, after_desc)) in enumerate(zip(before_list, after_list)):
                is_equal, diff = self._compare_dicts(before_dict, after_dict)
                if not is_equal:
                    diff_msg = f"{dict_type.name} comparison failed"
                    if before_desc or after_desc:
                        diff_msg += f" (Before: {before_desc}, After: {after_desc})"
                    diff_msg += f": {diff}"
                    differences.append(diff_msg)

        return differences
    
    def print_collections(self):
        """
        Print a formatted view of all collections before and after upgrade.
        """
        print("\nCollections Summary:")
        print("=" * 80)

        for dict_type in DictType:            
            before_list = self.before_upgrade[dict_type]
            after_list = self.after_upgrade[dict_type]

            if not before_list and not after_list:
                continue

            print(f"\n{dict_type.name}:")
            print("-" * 40)

            print("\nBefore upgrade:")
            if not before_list:
                print("  No collections")
            for i, (data, desc) in enumerate(before_list, 1):
                print(f"  Collection {i}" + (f" ({desc})" if desc else ""))
                if isinstance(data, dict):
                    for key, value in data.items():
                        print(f"    {key}: {value}")
                else:
                    print(f"    {data}")

            print("\nAfter upgrade:")
            if not after_list:
                print("  No collections")
            for i, (data, desc) in enumerate(after_list, 1):
                print(f"  Collection {i}" + (f" ({desc})" if desc else ""))
                if isinstance(data, dict):
                    for key, value in data.items():
                        print(f"    {key}: {value}")
                else:
                    print(f"    {data}")

            print("\n" + "=" * 80)

# SCENARIOS

Enter farm

In [ ]:
user_index = 1
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Enter farm with previous position

In [ ]:
user_index = 2
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_for_user_consolidated(user)
advance_blocks(5)
sleep(2)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_for_user_consolidated(user)
advance_blocks(5)
sleep(2)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Exit farm

In [ ]:
user_index = 2
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = exit_farm_user(user)
advance_blocks(5)
sleep(2)
exit_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, exit_op_1, "Initial exit farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = exit_farm_user(user)
advance_blocks(5)
sleep(2)
exit_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, exit_op_1, "Initial exit operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Claim rewards 

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Claim rewards multiple positions

In [ ]:
user_index = 2
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)

enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm operation")

tx_hash = claim_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, claim_op_1, "Initial claim farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)

enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm operation")

tx_hash = claim_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, claim_op_1, "Initial claim operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Claim boosted rewards

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_boosted_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim boosted farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_boosted_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim boosted operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Merge positions

In [ ]:
user_index = 2
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm operation")

tx_hash = merge_positions(user)
advance_blocks(5)
sleep(2)
merge_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, merge_op_1, "Initial merge positions operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter operation")

tx_hash = merge_positions(user)
advance_blocks(5)
sleep(2)
merge_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, merge_op_1, "Initial merge positions operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Enter farm on behalf

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
whitelisted_user = users_init()[1]

initial_blocks = 10
advance_blocks(initial_blocks)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [whitelisted_user.address.bech32()])
advance_blocks(5)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf(whitelisted_user, user)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
whitelisted_user = users_init()[1]

advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [whitelisted_user.address.bech32()])
advance_blocks(5)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf(whitelisted_user, user)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Enter farm on behalf existing position

In [ ]:
user_index = 2
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
whitelisted_user = users_init()[1]

initial_blocks = 10
advance_blocks(initial_blocks)

transfer_position(user, whitelisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [whitelisted_user.address.bech32()])
advance_blocks(5)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf(whitelisted_user, user)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
whitelisted_user = users_init()[1]

advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

transfer_position(user, whitelisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [whitelisted_user.address.bech32()])
advance_blocks(5)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf(whitelisted_user, user)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Enter farm on behalf blacklisted

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
blacklisted_user = users_init()[2]

initial_blocks = 10
advance_blocks(initial_blocks)

transfer_position(user, blacklisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_blacklist(user, context.network_provider.proxy, [blacklisted_user.address.bech32()])
advance_blocks(5)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf(blacklisted_user, user)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
blacklisted_user = users_init()[1]

advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

transfer_position(user, blacklisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_blacklist(user, context.network_provider.proxy, [blacklisted_user.address.bech32()])
advance_blocks(5)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf(blacklisted_user, user)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Claim rewards on behalf

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
blacklisted_user = users_init()[2]

initial_blocks = 10
advance_blocks(initial_blocks)

transfer_position(user, blacklisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [blacklisted_user.address.bech32()])
advance_blocks(5)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_rewards_on_behalf(blacklisted_user, user)
advance_blocks(5)
sleep(6)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim farm on behalf farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
blacklisted_user = users_init()[1]

advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

transfer_position(user, blacklisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [blacklisted_user.address.bech32()])
advance_blocks(5)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_rewards_on_behalf(blacklisted_user, user)
advance_blocks(5)
sleep(6)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Claim rewards on behalf blacklisted

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
blacklisted_user = users_init()[2]

initial_blocks = 10
advance_blocks(initial_blocks)

transfer_position(user, blacklisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_blacklist(user, context.network_provider.proxy, [blacklisted_user.address.bech32()])
advance_blocks(5)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_rewards_on_behalf(blacklisted_user, user)
advance_blocks(5)
sleep(6)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim farm on behalf farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
blacklisted_user = users_init()[1]

advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

transfer_position(user, blacklisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_blacklist(user, context.network_provider.proxy, [blacklisted_user.address.bech32()])
advance_blocks(5)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_rewards_on_behalf(blacklisted_user, user)
advance_blocks(5)
sleep(6)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)